In [2]:
import pandas as pd
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Pipeline,
    PreTrainedModel,
    PreTrainedTokenizerBase,
    pipeline,
)
import os
tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [14]:
os.listdir('../../data/training_data')

['D10001.csv',
 'D20002.csv',
 'training_data.csv',
 'D20003.csv',
 'D20001.csv',
 'D30004.csv',
 'D30001.csv',
 'D30002.csv',
 'D30003.csv',
 'clem_top_10_models_data_all_successful_episodes_no_preprocessing.csv',
 'clem_top_10_models_data_all_successful_episodes_individual_turns_adapted_imagegame.csv',
 '.ipynb_checkpoints',
 'clem_top_10_models_data_all_successful_episodes_individual_turns.csv',
 'D50001.csv',
 'D50003.csv',
 'D50002.csv',
 'D50006.csv',
 'D50007.csv',
 'D50005.csv',
 'D50004.csv']

In [15]:
all_datasets = ['D10001.csv',
                'D20002.csv',
                'D20003.csv',
                'D20001.csv',
                'D30004.csv',
                'D30001.csv',
                'D30002.csv',
                'D30003.csv',
                'D50001.csv',
                'D50003.csv',
                'D50002.csv',
                'D50006.csv',
                'D50007.csv',
                'D50005.csv',
                'D50004.csv']

In [20]:
for dataset in all_datasets:
    path = os.path.join('../../data/training_data', dataset)
    df = pd.read_csv(path)
    df['chat'] = df.chat.map(lambda x: eval(x))
    df["text"] = df.chat.map(lambda x: tokenizer.apply_chat_template(conversation=x, tokenize=False))
    df.to_csv(path, index=False)